In [2]:
table=['IR_BEN_R', 'IR_IMB_R', 'IR_IBA_R']
lib=["Table du Référentiel des bénéficiaires","Table du Référentiel médicalisé", "Table du Référentiel des bénéficiaires du DCIRS"]
tableexist={
    'tables':table,
    'tab_lib':lib,
    'produit':"Référentiels"
    }

In [1]:
pip install tableschema

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 6.7 MB/s 
     |████████████████████████████████| 41 kB 783 kB/s 
     |████████████████████████████████| 72 kB 566 kB/s 
     |████████████████████████████████| 126 kB 62.8 MB/s 
     |████████████████████████████████| 132 kB 47.8 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 9.3 MB 54.4 MB/s 
     |████████████████████████████████| 127 kB 73.9 MB/s 
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-py3-none-any.whl size=7400 sha256=cb863c89b11dbc48ad397796ad2e3a8264ab147bd726bb13e52b54b419b10b5d
  Stored in directory: /root/.cache/pip/wheels/dd/0a/62/49ee84fab92057fb3e303fb1a59e75ccac6899b417f441305a
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10768 sha256=149cd813da09aa101a58223c112daa309e56b64543b023ced3632e5b7cad14f8
  Stored in directory

In [14]:

from tableschema import Schema
import json
for table in tableexist['tables']:
  with open('{}.json'.format(table),'w', encoding='utf-8') as f:
    json.dump([],f, indent=4,ensure_ascii=False)
  import pandas as pd 
  df= pd.read_csv("{}.csv".format(table), sep=",",encoding='utf-8',na_values="")
  df.rename(columns={'Nom variable':'variables','Remarque':'observation','Libellé':'lib','Taille':'length'}, inplace=True)
  df=df.fillna('')


  ##### transformation des types 
  mapping_t = {
    "Char": "string",
    "Num":"integer",
    "Date":"date"
  }
  df["type"]=df["Type"].map(mapping_t)
  # Fonction qui permet de classer le type en number ou integer en fct de la taille 
  def numbint(x): 
    res = re.search(",", x)
    if res:
      return "number"
    return "integer"
  df["datecreated"]=''
  df["datedeleted"]=''
  df["datemissing"]=''
  
  df['nomenclature']=''
  df["rdg"]=''
  df['table']=table
  


 # Creation dy type oracle 
  mapping_to = {
    "string": "string",
    "date":"date",
    "integer":"number"
    
    }

  df["type_oracle"]=df["type"].map(mapping_to)

 
# Creation des fichiers json 
  for i in range(len(df['variables'])):
    if df['type'][i]=='number':
      df['type'][i]=df['length'][i].apply(numbint)    
     # les colonnes manquante  
  df=df.fillna('')
  
# create json file with the tables name
  if table is not 'IR_IBA_R':
    for z in range(len(df['variables'])): 
      new_dict={
              "name": df["variables"][z],
              "description": df["lib"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["datecreated"][z]),
              "dateDeleted": str(df["datedeleted"][z]),
              "dateMissing": df["datemissing"][z],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
      
      with open('{}.json'.format(table),encoding='utf-8') as f:  
        data=json.load(f)
        data.append(new_dict)
      with open('{}.json'.format(table), 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    # Mettons les fichiers sous format tableschema
  else:
    df['observation']=''
    for z in range(len(df['variables'])): 
      new_dict={
              "name": df["variables"][z],
              "description": df["lib"][z],
              "type": df["type"][z],
              "nomenclature": df["nomenclature"][z],
              "length": str(df["length"][z]),
              "format": "default",
              "dateCreated": str(df["datecreated"][z]),
              "dateDeleted": str(df["datedeleted"][z]),
              "dateMissing": df["datemissing"][z],
              "observation": df["observation"][z],
              "regle_gestion": df["rdg"][z],
              "type_oracle": df["type_oracle"][z]
        
          }
      
      with open('{}.json'.format(table),encoding='utf-8') as f:  
        data=json.load(f)
        data.append(new_dict)
      with open('{}.json'.format(table), 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
from tableschema import Schema
for table in tableexist['tables']:
  with open('{}.json'.format(table), "r", encoding='utf-8') as f:
    descriptor = {'fields': json.load(f)}
    schema=Schema(descriptor) 
    schema.descriptor['name']=table
    schema.descriptor['title']=tableexist['tab_lib'][tableexist['tables'].index(table)]
    schema.descriptor['produit']=tableexist['produit']
    schema.descriptor['missingValues'] = ['']   
    if table=='IR_BEN_R':
      schema.descriptor['primaryKey']=[
        "BEN_NIR_PSA",
        "BEN_RNG_GEM"
    ]
    if table=='IR_IBA_R':
      schema.descriptor['primaryKey']=['BEN_IDT_ANO']
      schema.descriptor['foreignKeys'] =[
        {
            "fields": [
                "BEN_IDT_ANO"
            ],
            "reference": {
                "resource": "IR_BEN_R",
                "fields": [
                    "BEN_IDT_ANO"
                ]
            },
            "description": "Bidirectionnel"
        }
    ]
    if table=='IR_IMB_R':
      schema.descriptor['foreignKeys']=[
        {
            "fields": [
                "BEN_NIR_PSA",
                "BEN_RNG_GEM"
            ],
            "reference": {
                "resource": "IR_BEN_R",
                "fields": [
                    "BEN_NIR_PSA",
                    "BEN_RNG_GEM"
                ]
            }
        }
    ]
    schema.descriptor['history']={"dateCreated": "",
                                    "dateDeleted": "",
                                    "dateMissing": []}
    schema.commit()
    schema.save('{}.json'.format(table),ensure_ascii=False)